In [1]:
import json
from pathlib import Path
from google import genai
from google.genai import types

In [2]:
credentials_path = Path("../credentials/credentials.json")

with credentials_path.open("r", encoding="utf-8") as file:
    credentials = json.load(file)

GEMINI_API_KEY      = credentials["gemini_api_key"]
GOOGLE_CHAT_API_KEY = credentials["google_chat_api_key"]

In [3]:
prompt_path = Path("../prompts/stocks_prompt.md")

with prompt_path.open("r", encoding="utf-8") as file:
    stocks_prompt = file.read()

In [4]:
client = genai.Client(
    api_key = GEMINI_API_KEY
)

config = types.GenerateContentConfig(
    tools = [
        types.Tool(
            google_search = types.GoogleSearch()
        )
    ]
)

response = client.models.generate_content(
    model    = "gemini-2.5-pro",
    contents = stocks_prompt.format(ticker_code="RAIZ4"),
    config   = config,
)

In [5]:
response.text

'## Market Intelligence Research: Raízen S.A. (RAIZ4)\n\n**Executive Summary:** Raízen S.A. (RAIZ4) is currently facing a challenging market environment characterized by a strong downward price trend, negative investor sentiment, and significant concerns regarding its financial health, particularly its high debt levels. Recent news includes a credit rating downgrade and analyst downgrades, alongside a widening net loss in the latest quarter. While the company has taken steps to manage its debt and has a history of dividend payments, the short-term outlook appears negative. Key drivers influencing the stock are its significant debt burden, pressures on profitability, and broader macroeconomic conditions in Brazil, including interest rate and commodity price fluctuations.\n\n### 1. Recent Price Behavior and Trend\n\nThe short-term trend for RAIZ4 is a **strong downward movement**. Over the past year, the stock has experienced a significant decline of approximately 57-59%. In the last few

In [6]:
response_2 = client.models.generate_content(
    model    = "gemini-2.5-flash-lite",
    contents = f"Faça um resumo em português: {response.text}"
)

In [7]:
response_2.text

'Aqui está um resumo em português do documento sobre a Raízen S.A. (RAIZ4):\n\n**Resumo Executivo:** A Raízen S.A. (RAIZ4) enfrenta atualmente um ambiente de mercado desafiador, caracterizado por uma forte tendência de queda nos preços, sentimento negativo dos investidores e preocupações significativas em relação à sua saúde financeira, especialmente seus altos níveis de endividamento. Notícias recentes incluem um rebaixamento de rating de crédito e revisões negativas por analistas, juntamente com um aumento na perda líquida no último trimestre. Embora a empresa tenha tomado medidas para gerenciar sua dívida e tenha um histórico de pagamento de dividendos, a perspectiva de curto prazo parece negativa. Os principais impulsionadores que influenciam a ação são seu significativo ônus da dívida, pressões sobre a lucratividade e as condições macroeconômicas mais amplas no Brasil, incluindo flutuações nas taxas de juros e preços das commodities.\n\n**1. Comportamento e Tendência Recente do Pr